In [ ]:
/*
Title:      FIT5202 Assessment 2 Phase 3 Group 1
Author:     Marcus Hudson, Chee Wong, Ertan Yesilnacar, Mike Abecina
Student ID: 30555094
Due Date:   09/06/2020
Task:       Data Wrangling of 2018 airbnb listings (Sydney) and Machine Learning Pipeline
Source:     listings_dec18.csv
Output:     Assessment_2_Wrangled.csv
*/

In [1]:
import scala.io.Source
import java.io.BufferedWriter
import java.io._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.ml.regression.{LinearRegression, LinearRegressionModel}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.{ StringIndexer, StringIndexerModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.{OneHotEncoderEstimator, StringIndexer}
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.{CrossValidator, CrossValidatorModel}
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.log4j.Logger
import org.apache.log4j.Level
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.sql.Row
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.{ StringIndexer, StringIndexerModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.functions.when

// Set log level to ERROR (less verbose)
sc.setLogLevel("ERROR")

Intitializing Scala interpreter ...

Spark Web UI available at http://c733cb369a3b:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591599937007)
SparkSession available as 'spark'


import scala.io.Source
import java.io.BufferedWriter
import java.io._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.ml.regression.{LinearRegression, LinearRegressionModel}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.{StringIndexer, StringIndexerModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.{OneHotEncoderEstimator, StringIndexer}
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.{CrossValidator, CrossValidatorModel}
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.log4j.Logger
import org.a...

## Data Wrangling and Transformation

In [2]:
// Load Listings
val rawArrayLength:Int = 1000 //36662 // hard-coded due to problem counting records in the file, caused by end of line
val totalColumns = 95
val rawArray = Array.ofDim[String](rawArrayLength, totalColumns) // Raw array to import source data
var count = 0
val fileName = "listings_dec18.csv"

// Load each line from the file and remove commas inside double-quoted text fields
for (line <- Source.fromFile(fileName, "UTF-8").getLines.drop(1).take(rawArrayLength)) { // Ignore header row .take(5000)
    var count_2 = -1
    if (count%1000 == 0) println(count)
    //println(line)
    line.split(",(?=([^\"]*\"[^\"]*\")*[^\"]*$)").foreach(colValue=> {count_2 += 1; rawArray(count)(count_2) = colValue}) // Don't split commas inside quotes
    count += 1
}

0


rawArrayLength: Int = 1000
totalColumns: Int = 95
rawArray: Array[Array[String]] = Array(Array(12351, https://www.airbnb.com/rooms/12351, 2.01812E+13, 12/7/2018, Sydney City & Harbour at the door, Come stay with Vinh & Stuart (Awarded as one of Australia's top hosts by Airbnb CEO Brian Chesky & key shareholder Ashton Kutcher. We're Sydney's #1 reviewed hosts too). Find out why we've been positively reviewed 500+ times. Message us and talk first BEFORE you make any reservation request - And please read our listing to the end (hint hint). Everything you need to know is there., "We're pretty relaxed hosts, and we fully appreciate staying with someone else, in their home home, is not for every-one. This is not a business, or a hotel. We're casual Airbnb hosts, not hoteliers. If you're just ...

In [3]:
val colListingID = 0
val colHostResponseRate = 25
val colCity = 40
val colZipCode = 42
val colPropertyType = 50
val colRoomType = 51
val colAccomodates = 52
val colBathrooms = 53
val colBedrooms = 54
val colBeds = 55
val colBedType = 56
val colAmenities = 57
val colPrice = 59
val colAvailability365 = 73
val colNumberofReviews = 75
val colReviewsScoreRating = 78 
val colCancellationPolicy = 90

colListingID: Int = 0
colHostResponseRate: Int = 25
colCity: Int = 40
colZipCode: Int = 42
colPropertyType: Int = 50
colRoomType: Int = 51
colAccomodates: Int = 52
colBathrooms: Int = 53
colBedrooms: Int = 54
colBeds: Int = 55
colBedType: Int = 56
colAmenities: Int = 57
colPrice: Int = 59
colAvailability365: Int = 73
colNumberofReviews: Int = 75
colReviewsScoreRating: Int = 78
colCancellationPolicy: Int = 90


In [4]:
// Cleanup any fail records caused by end of line problem ("\n") and extract an exact count of clean records
var fails = 0
for (i <- 0 to rawArrayLength - 1) {
    val x = rawArray(i)(0)
    if (!x.forall(_.isDigit)) {
        fails += 1
    }
}
println("Total eol fail records: " + fails)

// Move raw data to cleaned listings array
val listingsArrayLength = 36662
var listingsArrayToPos = -1
val listingsArray = Array.ofDim[String](listingsArrayLength, 95)
var cleanRecords = 0
for (i <- 0 to rawArrayLength - 1) {
    val x = rawArray(i)(0)
    if (x.forall(_.isDigit)) {
        listingsArrayToPos += 1
        if (listingsArrayToPos < 36662) {
            cleanRecords += 1
            listingsArray(listingsArrayToPos) = rawArray(i)
        }
    }
}

Total eol fail records: 0


fails: Int = 0
listingsArrayLength: Int = 36662
listingsArrayToPos: Int = 999
listingsArray: Array[Array[String]] = Array(Array(12351, https://www.airbnb.com/rooms/12351, 2.01812E+13, 12/7/2018, Sydney City & Harbour at the door, Come stay with Vinh & Stuart (Awarded as one of Australia's top hosts by Airbnb CEO Brian Chesky & key shareholder Ashton Kutcher. We're Sydney's #1 reviewed hosts too). Find out why we've been positively reviewed 500+ times. Message us and talk first BEFORE you make any reservation request - And please read our listing to the end (hint hint). Everything you need to know is there., "We're pretty relaxed hosts, and we fully appreciate staying with someone else, in their home home, is not for every-one. This is not a business, or a hotel. We're casual Airbnb host...

In [5]:
// Create an array of review scores based on postcode, property_type and bedrooms that can be used to fill blank review scores
var reviewArray = Array.ofDim[String](3000, 3) // columns are key, sum, count
var reviewItems = 0
for (rec <- 0 to cleanRecords - 1) {
    val zipCode = listingsArray(rec)(colZipCode)
    val propertyType = listingsArray(rec)(colPropertyType)
    val bedrooms = listingsArray(rec)(colBedrooms)
    val reviewsScoreRating = listingsArray(rec)(colReviewsScoreRating)
    if (zipCode != "" && propertyType != "" && bedrooms != "" && reviewsScoreRating != "") {
        val key = zipCode + propertyType + bedrooms
        var key_found = false
        for (revRec <- 0 to reviewItems) {
            if (reviewArray(revRec)(0) == key) { // Key matched, update key record sum and count
                val existingSum = reviewArray(revRec)(1).toInt
                val count = reviewArray(revRec)(2).toInt
                reviewArray(revRec)(1) = (existingSum+reviewsScoreRating.toInt).toString
                reviewArray(revRec)(2) = (count+1).toString
                key_found = true
            }
        }
        if (!key_found) { // Create array record when no matching key can be found
                reviewArray(reviewItems)(0) = key
                reviewArray(reviewItems)(1) = reviewsScoreRating
                reviewArray(reviewItems)(2) = "1"
                reviewItems += 1
        }
    }
}

reviewArray: Array[Array[String]] = Array(Array(2009Townhouse1, 191, 2), Array(2093House3, 296, 3), Array(2010Apartment1, 2050, 22), Array(2041House4, 179, 2), Array(2023Apartment1, 358, 4), Array(2060Apartment0, 97, 1), Array(2026Guest suite1, 87, 1), Array(2010Loft1, 476, 5), Array(2026House5, 95, 1), Array(2026Apartment2, 4705, 52), Array(2026Apartment1, 4321, 46), Array(2088House1, 190, 2), Array(2015House4, 93, 1), Array(2107Guesthouse0, 97, 1), Array(2066House1, 383, 4), Array(2015House1, 184, 2), Array(2021Apartment1, 722, 8), Array(2011Apartment1, 2705, 29), Array(2204House2, 196, 2), Array(2155Villa1, 100, 1), Array(2126House2, 98, 1), Array(2031Apartment2, 371, 4), Array(2034Villa5, 98, 1), Array(2036Apartment1, 98, 1), Array(2031House1, 389, 4), Array(2011Apartment3, 89, 1), ...

In [6]:
// Create an array of city and zip to fill blank city and zip

var zipArray = Array.ofDim[String](1000, 4) // columns are key, zipcode, city, count
var zipItems = 0
for (rec <- 0 to cleanRecords - 1) {
    val city = listingsArray(rec)(colCity)
    val zipCode = listingsArray(rec)(colZipCode)
    if (zipCode != "" && city != "") {
        val key = zipCode + city
        var key_found = false
        for (zipRec <- 0 to zipItems) {
            if (zipArray(zipRec)(0) == key) { // Key matched, update key record count
                val existingCount = zipArray(zipRec)(3).toInt
                zipArray(zipRec)(3) = (existingCount+1).toString
                key_found = true
            }
        }
        if (!key_found) { // Create array record when no matching key can be found
            zipArray(zipItems)(0) = key
            zipArray(zipItems)(1) = zipCode
            zipArray(zipItems)(2) = city
            zipArray(zipItems)(3) = "1"
            zipItems += 1
        }
    }
}

zipArray: Array[Array[String]] = Array(Array(2009Pyrmont, 2009, Pyrmont, 10), Array(2093Balgowlah, 2093, Balgowlah, 7), Array(2010Darlinghurst, 2010, Darlinghurst, 34), Array(2041Balmain, 2041, Balmain, 11), Array(2023Bellevue Hill, 2023, Bellevue Hill, 13), Array(2060North Sydney, 2060, North Sydney, 9), Array(2026North Bondi, 2026, North Bondi, 37), Array(2026Bondi Beach, 2026, Bondi Beach, 64), Array(2088Mosman, 2088, Mosman, 13), Array(2022Bondi Junction, 2022, Bondi Junction, 19), Array(2015Alexandria, 2015, Alexandria, 7), Array(2107Avalon, 2107, Avalon, 13), Array(2026Sydney, 2026, Sydney, 5), Array(2066Lane Cove West, 2066, Lane Cove West, 1), Array(2021Paddington, 2021, Paddington, 15), Array(2011Elizabeth Bay, 2011, Elizabeth Bay, 6), Array(2204Marrickville, 2204, Marrickville...

In [7]:
// Wrangle Listings and Write output file of wrangled data
// Process the list of chosen column id's and if a column requires specific transformation, it will be listed in the main loop

val file = new File("Assessment_2_Wrangled.csv")
val bw = new BufferedWriter(new FileWriter(file))
val propertyTypeHouse = List("apartment","bungalow","cabin","casa particular (cuba)","chalet","condominium","cottage","dome house","earth house","farm stay","guest suite","guesthouse","house","tiny house","townhouse","villa")
val propertyTypeHotel = List("aparthotel","bed and breakfast","boutique hotel","heritage hotel (india)","hostel","hotel","resort","serviced apartment")
val BedTypeList = List("real bed")
var discardResponse = 0
var discardWeirdCityChar = 0
var discardNoCity = 0
var discardNoZip = 0
var discardNoAvailability = 0
var discardNoScore = 0
val colsToWrite = List(colListingID, colHostResponseRate, colCity, colZipCode, colPropertyType, colRoomType, colAccomodates, colBathrooms, colBedrooms, colBeds, colBedType, colAmenities, colPrice, colAvailability365, colNumberofReviews, colReviewsScoreRating, colCancellationPolicy)
    val colHeaders = "listing_id,host_response_rate,city,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,wifi,kitchen,washer,cable_tv,price,availability_365,number_of_reviews,review_scores_rating,cancellation_policy,estimated_annual_revenue,no_bedrooms,amenities\n"
val ordinary=(('a' to 'z') ++ ('A' to 'Z') ++ ('0' to '9') ++ (' ' to '/')).toSet  // To help filter out weird city characters
def isOrdinary(s:String)=s.forall(ordinary.contains(_)) // To help filter out weird city characters

bw.write(colHeaders) // Write header to output file
for (rec <- 0 to cleanRecords - 1) { // Loop through every record in the clean listings array
    // Set default values
    var price = 0.00
    var available365 = 0
    var outLine:String = "" // Line to write to output file
    var WriteRecord = true
    var propertyType = ""
    var bedRooms = ""
    var Accomodates = ""
    var ListingID = ""
    var bathroom_tmp = 0
    var no_bedrooms = 0
    var newAmenities = ""
    
    for (col <- 0 to totalColumns - 1) { // Loop through each column in the listing row
        if (colsToWrite.contains(col)) { // Only process columns that we have chosen for output
            if (col == colListingID) ListingID = listingsArray(rec)(col) // Listing ID
            if (col == colHostResponseRate) { // host_response_rate
                if (listingsArray(rec)(col) != "N/A" & listingsArray(rec)(col) != "") { // Discard if "NA" or NULL
                    val hostResponseRate = listingsArray(rec)(col).replace("%","") // Remove percentage sign
                    outLine = outLine + "," + hostResponseRate
                }
                else {
                    if (WriteRecord) discardResponse += 1
                    WriteRecord = false // Discard records with "N/A" or NULL
                }
            }
            else if (col == colCity) { // city
                var city = listingsArray(rec)(col)
                if (!isOrdinary(city)) {  // Discard records with weird characters
                    if (WriteRecord) discardWeirdCityChar += 1
                    WriteRecord = false
                }
                else if (city == "") {
                    val zipCode = listingsArray(rec)(colZipCode)
                    if (zipCode != "") {
                        var maxCount = 0
                        for (n <- 0 to zipItems - 1) {
                            val ArrayZip = zipArray(n)(1)
                            val ArrayCity = zipArray(n)(2)
                            val ArrayCount = zipArray(n)(3).toInt
                            if (ArrayZip == zipCode) {
                                if (count.toInt > maxCount) {
                                    city = ArrayCity
                                    maxCount = ArrayCount
                                }
                            }
                        }
                    }
                    if (city == "") {
                        if (WriteRecord) discardNoCity += 1
                        WriteRecord = false // Discard records where we can;t find the city
                    }
                }
                if (WriteRecord) outLine = outLine + "," + city
            }
            else if (col == colZipCode) { // zipcode
                var zipCode = listingsArray(rec)(col)
                if (zipCode == "") {
                    val city = listingsArray(rec)(colCity)
                    if (city != "") {
                        var maxCount = 0
                        for (n <- 0 to zipItems - 1) { // Use the city/zip array to find the most common match
                            val Arrayzip = zipArray(n)(1)
                            val ArrayCity = zipArray(n)(2)
                            val ArrayCount = zipArray(n)(3).toInt
                            if (ArrayCity == city) {
                                if (count.toInt > maxCount) {
                                    zipCode = Arrayzip
                                    maxCount = ArrayCount
                                }
                            }
                        }
                    }
                    if (zipCode == "") {
                        if (WriteRecord) discardNoZip += 1
                        WriteRecord = false // Discard records where we can't find the zipcode
                    }
                }
                if (zipCode.length > 4) zipCode = zipCode.substring(zipCode.length-4, zipCode.length) // Final zipcode cleanup
                if (WriteRecord) outLine = outLine + "," + zipCode
            }
            else if (col == colPropertyType) { // property_type
                propertyType = listingsArray(rec)(col).toLowerCase() // Convert property type to 1 of 3 nominal values
                if (propertyTypeHotel.contains(propertyType)) {
                    outLine = outLine + ",hotel"
                }
                else if (propertyTypeHouse.contains(propertyType)) {
                    outLine = outLine + ",house"
                }
                else {
                    outLine = outLine + ",other"
                }
            }
            else if (col == colAccomodates) { // accommodates
                Accomodates = listingsArray(rec)(col)
                outLine = outLine + "," + Accomodates
            }
            else if (col == colBathrooms) { // bathrooms
                var bathRooms = listingsArray(rec)(col)
                if (bathRooms == "") {
                    if (propertyTypeHotel.contains(propertyType)) bathRooms = "1" // Set empty bathrooms to 1 for hotel
                    else if (propertyTypeHouse.contains(propertyType)) bathRooms = "1" // Set empty bathrooms to 1 for house
                    else bathRooms = "0" // Set empty bathrooms to 0 for other property type
                }
                // Temporary calculated column bathroom_tmp
                val bathInt = bathRooms.toFloat
                if (bathInt > 2) bathroom_tmp = 3
                else if (bathInt > 1) bathroom_tmp = 2
                else if (bathInt > 0) bathroom_tmp = 1
                else bathroom_tmp = 0

                outLine = outLine + "," + bathRooms
            }
            else if (col == colBedrooms) { // bedrooms
                bedRooms = listingsArray(rec)(col)
                if (bedRooms == "") {
                    val beds = listingsArray(rec)(col+1)
                    if (beds != "") bedRooms = beds // Set empty bedrooms to number of beds
                    else bedRooms = Accomodates // Set empty bedrooms to number of people that can be accomodated
                }
                // Calculated column no_bedrooms
                val bedInt = bedRooms.toInt
                if (bedInt > 3) no_bedrooms = 4
                else if (bedInt > 2) no_bedrooms = 3
                else if (bedInt > 1) no_bedrooms = 2
                else no_bedrooms = 1

                outLine = outLine + "," + bedRooms
            }
            else if (col == colBeds) { // beds
                var beds = listingsArray(rec)(col)
                if (beds == "") {
                    if (bedRooms != "") {
                        beds = bedRooms // Set empty beds to number of bedrooms
                    }
                }
                outLine = outLine + "," + beds
            }   
            else if (col == colBedType) { // bed_type
                val bedType = listingsArray(rec)(col).toLowerCase()  // Convert bed type to 1 of 2 nominal values
                if (BedTypeList.contains(bedType)) {
                    outLine = outLine + ",real bed"
                }
                else {
                    outLine = outLine + ",other"
                }
            }
            else if (col == colAmenities) { // amenities
                val amenities = listingsArray(rec)(col).toLowerCase()
                
                // New calculated amenities column
                newAmenities = bathroom_tmp.toString
                
                // Ordinal: Check amenities string for wifi, kitchen, washer and cable tv and set to 1 if found, otherwise 0
                if (amenities.contains("wifi")) {
                    outLine = outLine + ",1"
                }
                else outLine = outLine + ",0"
                if (amenities.contains("kitchen")) {
                    outLine = outLine + ",1"
                }
                else outLine = outLine + ",0"
                if (amenities.contains("washer")) {
                    outLine = outLine + ",1"
                }
                else outLine = outLine + ",0"
                if (amenities.contains("cable tv")) {
                    outLine = outLine + ",1"
                }
                else outLine = outLine + ",0"

                if (amenities.contains("kitchen")) newAmenities += "1" else newAmenities += "0"
                if (amenities.contains("wifi")) newAmenities += "1" else newAmenities += "0"
            }
            else if (col == colPrice) { // price
                // Remove unwanted characters from the price string to transform to a double
                var priceVal = listingsArray(rec)(col).toString.replace(",","").replace("$","").replace("\"", "").trim()
                outLine = outLine + "," + priceVal
                price = priceVal.toDouble
            }
            else if (col == colAvailability365) { // availability_365
                val newVal = listingsArray(rec)(col)
                if (newVal != "0") { // Discard listings that have no availability
                    outLine = outLine + "," + newVal
                    available365 = newVal.toInt
                }
                else {
                    if (WriteRecord) discardNoAvailability += 1
                    WriteRecord = false
                }
            }
            else if (col == colReviewsScoreRating) { // reviews_score_rating
                var score = listingsArray(rec)(col)
                val reviews = listingsArray(rec)(colNumberofReviews)

                if (score == "") {
                    if (reviews == "0") score = "0" // Set empty review score to 0 where the listing has 0 reviews
                    else { // Use the average review score array to match by property type, zipcode and number of bedrooms
                        val zipCode = listingsArray(rec)(colZipCode)
                        val propertyType = listingsArray(rec)(colPropertyType)
                        val bedrooms = listingsArray(rec)(colBedrooms)
                        val key = zipCode + propertyType + bedrooms
                        var keyFound = false
                        for (revRec <- 0 to reviewItems) {
                            if (reviewArray(revRec)(0) == key) {
                                val avg:Int = reviewArray(revRec)(1).toInt / reviewArray(revRec)(2).toInt
                                score = avg.toString
                                listingsArray(rec)(col) = score
                                keyFound = true
                            }
                        }
                        if (!keyFound) {
                            if (WriteRecord) discardNoScore += 1
                            WriteRecord = false  // Discard as there is no review information available
                        }
                    }
                }
                outLine = outLine + "," + score
            }
            else if (col == colCancellationPolicy) { // cancellation_policy
                var cancellationPolicy = listingsArray(rec)(col).toLowerCase()
                // Convert cancellation policies that begin with strict and super strict to strict
                if (cancellationPolicy.substring(0,6) == "strict" || cancellationPolicy.substring(0,5) == "super") cancellationPolicy = "strict"
                outLine = outLine + "," + cancellationPolicy
            }
            else { // Default for columns that require no transformation
                if (col != 0) {
                    outLine = outLine + ","
                }
                outLine = outLine + listingsArray(rec)(col)
            }
        }
    }
    val annualRevenue = (365 - available365) * price // Calculate annual revenue for the listing based on price and availability
    outLine += "," + f"$annualRevenue%1.2f"
    
    // Add extra calculated columns to the end of the line
    outLine += "," + no_bedrooms.toString + "," + newAmenities

    outLine += "\n"
    if (WriteRecord) bw.write(outLine)
}
bw.close()
println("Discard invalid response rate: " + discardResponse)
println("Discard weird city characters: " + discardWeirdCityChar)
println("Discard cannot determine city: " + discardNoCity)
println("Discard cannot determine zipcode: " + discardNoZip)
println("Discard listing has no availability: " + discardNoAvailability)
println("Discard cannot determine review score: " + discardNoScore)


Discard invalid response rate: 473
Discard weird city characters: 0
Discard cannot determine city: 0
Discard cannot determine zipcode: 0
Discard listing has no availability: 11
Discard cannot determine review score: 1


file: java.io.File = Assessment_2_Wrangled.csv
bw: java.io.BufferedWriter = java.io.BufferedWriter@3af7ac10
propertyTypeHouse: List[String] = List(apartment, bungalow, cabin, casa particular (cuba), chalet, condominium, cottage, dome house, earth house, farm stay, guest suite, guesthouse, house, tiny house, townhouse, villa)
propertyTypeHotel: List[String] = List(aparthotel, bed and breakfast, boutique hotel, heritage hotel (india), hostel, hotel, resort, serviced apartment)
BedTypeList: List[String] = List(real bed)
discardResponse: Int = 473
discardWeirdCityChar: Int = 0
discardNoCity: Int = 0
discardNoZip: Int = 0
discardNoAvailability: Int = 11
discardNoScore: Int = 1
colsToWrite: List[Int] = List(0, 25, 40, 42, 50, 51, 52, 53, 54, 55, 56, 57, 59, 73, 75, 78, 90)
colHeaders: String ...

## Machine Learning Pipeline

In [8]:
// Create new Spark session
object SparkSessionCreate {
  def createSession(): SparkSession = {
    val spark = SparkSession
      .builder
      .master("local[*]")
      .appName(s"FIT5202Asessment02Phase03")
      .getOrCreate()

    return spark
  }
}

defined object SparkSessionCreate


In [9]:
// Customer schema to force column type
val customSchema = StructType(
  Array(
    StructField("listing_id", StringType, true),
    StructField("host_response_rate", IntegerType, true),
    StructField("city", StringType, true),
    StructField("zipcode", StringType, true),
    StructField("property_type", StringType, true),
    StructField("room_type", StringType, true),
    StructField("accommodates", IntegerType, true),
    StructField("bathrooms", DoubleType, true),
    StructField("bedrooms", IntegerType, true),
    StructField("beds", IntegerType, true),
    StructField("bed_type", StringType, true),
    StructField("wifi", IntegerType, true),
    StructField("kitchen", IntegerType, true),
    StructField("washer", IntegerType, true),
    StructField("cable_tv", IntegerType, true),
    StructField("price", DoubleType, true),
    StructField("availability_365", IntegerType, true),
    StructField("number_of_reviews", IntegerType, true),
    StructField("review_scores_rating", IntegerType, true),
    StructField("cancellation_policy", StringType, true),
    StructField("estimated_annual_revenue", DoubleType, true),
    StructField("no_bedrooms", IntegerType, true),
    StructField("amenities", IntegerType, true)
  )
)

customSchema: org.apache.spark.sql.types.StructType = StructType(StructField(listing_id,StringType,true), StructField(host_response_rate,IntegerType,true), StructField(city,StringType,true), StructField(zipcode,StringType,true), StructField(property_type,StringType,true), StructField(room_type,StringType,true), StructField(accommodates,IntegerType,true), StructField(bathrooms,DoubleType,true), StructField(bedrooms,IntegerType,true), StructField(beds,IntegerType,true), StructField(bed_type,StringType,true), StructField(wifi,IntegerType,true), StructField(kitchen,IntegerType,true), StructField(washer,IntegerType,true), StructField(cable_tv,IntegerType,true), StructField(price,DoubleType,true), StructField(availability_365,IntegerType,true), StructField(number_of_reviews,IntegerType,true),...

## Linear Regression Model

### Preprocessing

In [10]:
object Preprocessing {
    // Shuffling and preparing the data
    var initialSample = 1.0

    //val spark = SparkSessionCreate.createSession()
    val wrangledData = "Assessment_2_Wrangled.csv"
    
    val spark = SparkSessionCreate.createSession()
    
    import spark.implicits._

    val dataInput = spark.read
        .option("header", "true")
        .schema(customSchema)
        .format("com.databricks.spark.csv")
        .load(wrangledData)

    
    // DATA ARRANGEMENT BEFORE THE SPLIT
    // ============================================
    
    println("Preparing data for before the training")
    val dataArrange1 = dataInput.withColumn("property_code", when(col("property_type") === "house", lit("3").cast(IntegerType))
          .when(col("property_type") === "hotel", lit("2").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange2 = dataArrange1.withColumn("room_code", when(col("room_type") === "Entire home/apt", lit("3").cast(IntegerType))
          .when(col("room_type") === "Private room", lit("2").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange3 = dataArrange2.withColumn("bed_type_code", when(col("bed_type") === "real bed", lit("3").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange4 = dataArrange3.withColumn("cancellation_code", when(col("cancellation_policy") === "strict", lit("1").cast(IntegerType))
          .when(col("cancellation_policy") === "moderate", lit("2").cast(IntegerType))
          .otherwise(lit("3").cast(IntegerType)))

    // filtering the data and remove rows we have defined in our EDA work
    // Step 01:
    val dataFilt = dataArrange4.filter(!(dataArrange4("no_bedrooms") === 1 && dataArrange4("amenities") === 111 && dataArrange4("price") > 7500))

    // Step 02:
    var amenitiesList = List(211, 311)
    val dataFiltered = dataFilt.filter(!(dataFilt("no_bedrooms") === 3 && dataFilt("amenities").isin(amenitiesList:_*) && dataFilt("price") > 5000))
    
    // Drop unnecessary fields
    val dataDrop = dataFiltered.drop("city", "listing_id","property_type", "room_type", "cable_tv", "bed_type", "price", "cancellation_policy")

    var data = dataDrop.withColumnRenamed("estimated_annual_revenue", "label").sample(false, initialSample)

    
    var DF = data.na.drop()

    // Null check
    if (data == DF)
    println("No null values in the DataFrame")

    else {
    println("Null values exist in the DataFrame")
    data = DF
    }

    println(data.printSchema())
    data.select("zipcode", "label", "property_code", "room_code", "no_bedrooms",  
                "accommodates", "amenities", "cancellation_code", "availability_365").show()
    
    // ============================================
    // TWO-STAGE SPLITTING 
    // ============================================
    
    // 1) Split the training data as train (training + validation) and test data
    val seed = 12345L
    val splits01 = data.randomSplit(Array(0.75, 0.25), seed)
    val (trainData, testData) = (splits01(0), splits01(1))               
    
    // 2) Split the train data as training and validation data
    val splits02 = trainData.randomSplit(Array(0.8, 0.2), seed)
    val (trainingData, validationData) = (splits02(0), splits02(1))

    // Caching all data for faster in-memory access
    trainingData.cache
    validationData.cache
    testData.cache
    
    // ============================================
    // DATA PIPELINE ARRANGEMENT
    // ============================================
    // Identify the categorical columns
    def isCateg(c: String): Boolean = c matches "zipcode"

    def categNewCol(c: String): String = if (isCateg(c)) s"idx_${c}" else c

    // Function to select only feature columns (omit id and label)
    def onlyFeatureCols(c: String): Boolean = !(c matches "id|label|zipcode")

    // Definitive set of feature columns
    val featureCols = trainingData.columns
        .filter(onlyFeatureCols)
        .map(categNewCol)

    // StringIndexer for categorical columns (OneHotEncoder should be evaluated as well)
    val stringIndexerStages = trainingData.columns.filter(isCateg)
      .map(c => new StringIndexer()
      .setInputCol(c)
      .setOutputCol(categNewCol(c))
      .fit(dataInput.select(c)))    
    
      // VectorAssembler for training features
      val assembler = new VectorAssembler()
        .setInputCols(featureCols)
        .setOutputCol("features")
    // ============================================    
}

defined object Preprocessing


### Run Pipeline

In [11]:
// Defining hyperparameters 
// number of folds for cross-validation

val numFolds = 10
// number of maximum iteration
val MaxIter: Seq[Int] = Seq(1000)
// regression parameter
val RegParam: Seq[Double] = Seq(0.001)
// the value of tolerance
val Tol: Seq[Double] = Seq(1e-6)
// finally, elastic network parameter
val ElasticNetParam: Seq[Double] = Seq(1)

// Create an LinerRegression estimator
val model = new LinearRegression()
    .setFeaturesCol("features")
    .setLabelCol("label")

// Building the Pipeline for transformations and predictor
println("Building ML pipeline")
val pipeline = new Pipeline().setStages(Preprocessing.stringIndexerStages :+ Preprocessing.assembler :+ model)

// ***********************************************************
println("Preparing K-fold Cross Validation and Grid Search: Model tuning")
// ***********************************************************
val paramGrid = new ParamGridBuilder()
  .addGrid(model.maxIter, MaxIter)
  .addGrid(model.regParam, RegParam)
  .addGrid(model.tol, Tol)
  .addGrid(model.elasticNetParam, ElasticNetParam)
  .build()

val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new RegressionEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(numFolds)

// ************************************************************
println("Training model with Linear Regression algorithm")
// ************************************************************
val cvModel = cv.fit(Preprocessing.trainingData)

// Save the workflow
cvModel.write.overwrite().save("model/LR_model")

// **********************************************************************
println("Evaluating model on train and validation set and calculating RMSE")
// **********************************************************************
val trainPredictionsAndLabels = cvModel.transform(Preprocessing.trainingData).select("label", "prediction")
  .map { case Row(label: Double, prediction: Double) => (label, prediction) }.rdd

val validPredictionsAndLabels = cvModel.transform(Preprocessing.validationData).select("label", "prediction")
  .map { case Row(label: Double, prediction: Double) => (label, prediction) }.rdd  

val trainRegressionMetrics = new RegressionMetrics(trainPredictionsAndLabels)
val validRegressionMetrics = new RegressionMetrics(validPredictionsAndLabels)
val bestModel = cvModel.bestModel.asInstanceOf[PipelineModel]

val results = "\n=====================================================================\n" +
  s"Param trainSample: ${Preprocessing.initialSample}\n" +
  s"Param testSample: ${Preprocessing.initialSample}\n" +
  s"TrainingData count: ${Preprocessing.trainingData.count}\n" +
  s"ValidationData count: ${Preprocessing.validationData.count}\n" +
  s"TestData count: ${Preprocessing.testData.count}\n" +
  "=====================================================================\n" +
  s"Param maxIter = ${MaxIter.mkString(",")}\n" +
  s"Param numFolds = ${numFolds}\n" +
  "=====================================================================\n" +
  s"Training data MSE = ${trainRegressionMetrics.meanSquaredError}\n" +
  s"Training data RMSE = ${trainRegressionMetrics.rootMeanSquaredError}\n" +
  s"Training data R-squared = ${trainRegressionMetrics.r2}\n" +
  s"Training data MAE = ${trainRegressionMetrics.meanAbsoluteError}\n" +
  s"Training data Explained variance = ${trainRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n" +
  s"Validation data MSE = ${validRegressionMetrics.meanSquaredError}\n" +
  s"Validation data RMSE = ${validRegressionMetrics.rootMeanSquaredError}\n" +
  s"Validation data R-squared = ${validRegressionMetrics.r2}\n" +
  s"Validation data MAE = ${validRegressionMetrics.meanAbsoluteError}\n" +
  s"Validation data Explained variance = ${validRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n\n" +
  s"CV params explained: ${cvModel.explainParams}\n" +
  s"GBT params explained: ${bestModel.stages.last.asInstanceOf[LinearRegressionModel].explainParams}\n" +
  "\n=====================================================================\n"
println(results)


    val model_columns = Preprocessing.featureCols
    val stages = bestModel.stages
    val coefficients = stages(2).asInstanceOf[LinearRegressionModel].coefficients.toArray
    val intercept = stages(2).asInstanceOf[LinearRegressionModel].intercept
    val coefficient_value = model_columns zip coefficients
    for (i<- 0 to 16){
        println(coefficient_value(i))
    }


// *****************************************
// Now see if we can predict values in our test data
// Generate predictions using our linear regression model for all features in our test data
println("Run prediction on the test set")

val fullPredictions = cvModel.transform(Preprocessing.testData).cache()

// This basically adds a "prediction" column to our testDF dataframe

// Extract the predictions and the "known" correct labels
val testPredictionsAndLabels = fullPredictions.select("prediction", "label").rdd.map(x => (x.getDouble(0), x.getDouble(1)))

// Print out the predicted and actual values for each point
/*
for (prediction <- testPredictionsAndLabels) {
  println(prediction)
}
*/

val testRegressionMetrics = new RegressionMetrics(testPredictionsAndLabels)
val testresults = "\n=====================================================================\n" +
  s"Test data MSE = ${testRegressionMetrics.meanSquaredError}\n" +
  s"Test data RMSE = ${testRegressionMetrics.rootMeanSquaredError}\n" +
  s"Test data R-squared = ${testRegressionMetrics.r2}\n" +
  s"Test data MAE = ${testRegressionMetrics.meanAbsoluteError}\n" +
  s"Test data Explained variance = ${testRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n"
println(testresults)

Building ML pipeline
Preparing data for before the training
Null values exist in the DataFrame
root
 |-- host_response_rate: integer (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- accommodates: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- beds: integer (nullable = true)
 |-- wifi: integer (nullable = true)
 |-- kitchen: integer (nullable = true)
 |-- washer: integer (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- review_scores_rating: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- no_bedrooms: integer (nullable = true)
 |-- amenities: integer (nullable = true)
 |-- property_code: integer (nullable = true)
 |-- room_code: integer (nullable = true)
 |-- bed_type_code: integer (nullable = true)
 |-- cancellation_code: integer (nullable = true)

()
+-------+--------+-------------+---------+-----------+------------+

numFolds: Int = 10
MaxIter: Seq[Int] = List(1000)
RegParam: Seq[Double] = List(0.001)
Tol: Seq[Double] = List(1.0E-6)
ElasticNetParam: Seq[Double] = List(1.0)
model: org.apache.spark.ml.regression.LinearRegression = linReg_35527320a16b
pipeline: org.apache.spark.ml.Pipeline = pipeline_0266634ba08d
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linReg_35527320a16b-elasticNetParam: 1.0,
	linReg_35527320a16b-maxIter: 1000,
	linReg_35527320a16b-regParam: 0.001,
	linReg_35527320a16b-tol: 1.0E-6
})
cv: org.apache.spark.ml.tuning.CrossValidator = cv_aa18056717b0
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_aa18056717b0
trainPredictionsAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[704] at rdd at <console>:110
validPredictionsAndLabels:...

## Log-Linear Regression Model

### Preprocessing

In [12]:
object Preprocessing {
    // Shuffling and preparing the data
    var initialSample = 1.0

    //val spark = SparkSessionCreate.createSession()
    val wrangledData = "Assessment_2_Wrangled.csv"
    
    val spark = SparkSessionCreate.createSession()
    
    import spark.implicits._

    val dataInput = spark.read
        .option("header", "true")
        .schema(customSchema)
        .format("com.databricks.spark.csv")
        .load(wrangledData)
    
    // DATA ARRANGEMENT BEFORE THE SPLIT
    // ============================================
    
    println("Preparing data for before the training")
    val dataArrange1 = dataInput.withColumn("property_code", when(col("property_type") === "house", lit("3").cast(IntegerType))
          .when(col("property_type") === "hotel", lit("2").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange2 = dataArrange1.withColumn("room_code", when(col("room_type") === "Entire home/apt", lit("3").cast(IntegerType))
          .when(col("room_type") === "Private room", lit("2").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange3 = dataArrange2.withColumn("bed_type_code", when(col("bed_type") === "real bed", lit("3").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange4 = dataArrange3.withColumn("cancellation_code", when(col("cancellation_policy") === "strict", lit("1").cast(IntegerType))
          .when(col("cancellation_policy") === "moderate", lit("2").cast(IntegerType))
          .otherwise(lit("3").cast(IntegerType)))
    
    // Taking natural logarithm of selected fields which are considered in our proposal and EDA work
    val dataLog1 = dataArrange4.withColumn("log_estimated_annual_revenue", log10("estimated_annual_revenue"))
    val dataLog2 = dataLog1.withColumn("log_accom", log10("accommodates"))
    val dataLog3 = dataLog2.withColumn("log_rating", log10("review_scores_rating"))   

    // Filtering the data and remove rows we have defined in our EDA work
    // Step 01:
    val dataFilt = dataLog3.filter(!(dataLog3("no_bedrooms") === 1 && dataLog3("amenities") === 111 && dataLog3("price") > 7500))

    // Step 02:
    var amenitiesList = List(211, 311)
    val dataFiltered = dataFilt.filter(!(dataFilt("no_bedrooms") === 3 && dataFilt("amenities").isin(amenitiesList:_*) && dataFilt("price") > 5000))

    
    // Drop unnecessary fields
    val dataDrop = dataFiltered.drop("property_type", "room_type", "cable_tv", "listing_id",
                                     "bed_type", "cancellation_policy", "kitchen", "wifi", "cable_tv",
                                     "host_response_rate", "bathrooms", "city", "estimated_annual_revenue",
                                     "accommodates", "beds", "price")
   
    var data = dataDrop.withColumnRenamed("log_estimated_annual_revenue", "label").sample(false, initialSample)
    
    var DF = data.na.drop()

    // Null check
    if (data == DF)
    println("No null values in the DataFrame")

    else {
    println("Null values exist in the DataFrame")
    data = DF
    }

    println(data.printSchema())
    data.select("zipcode", "label", "log_accom", "amenities", "log_rating", 
                "cancellation_code", "availability_365").show()
    
    // ============================================
    // TWO-STAGE SPLITTING 
    // ============================================
    
    // 1) Split the training data as train (training + validation) and test data
    val seed = 12345L
    val splits01 = data.randomSplit(Array(0.75, 0.25), seed)
    val (trainData, testData) = (splits01(0), splits01(1))               
    
    // 2) Split the train data as training and validation data
    val splits02 = trainData.randomSplit(Array(0.8, 0.2), seed)
    val (trainingData, validationData) = (splits02(0), splits02(1))

    // Caching all data for faster in-memory access
    trainingData.cache
    validationData.cache
    testData.cache
    
    // ============================================
    // DATA PIPELINE ARRANGEMENT
    // ============================================
    // Identify the categorical columns
    def isCateg(c: String): Boolean = c matches "zipcode"

    def categNewCol(c: String): String = if (isCateg(c)) s"idx_${c}" else c

    // Function to select only feature columns (omit id and label)
    def onlyFeatureCols(c: String): Boolean = !(c matches "id|label|zipcode")

    // Definitive set of feature columns
    val featureCols = trainingData.columns
        .filter(onlyFeatureCols)
        .map(categNewCol)

    // StringIndexer for categorical columns (OneHotEncoder should be evaluated as well)
    val stringIndexerStages = trainingData.columns.filter(isCateg)
      .map(c => new StringIndexer()
      .setInputCol(c)
      .setOutputCol(categNewCol(c))
      .fit(dataInput.select(c)))    
    
      // VectorAssembler for training features
      val assembler = new VectorAssembler()
        .setInputCols(featureCols)
        .setOutputCol("features")
    // ============================================     
}

defined object Preprocessing


### Run Pipeline

In [13]:
// Defining hyperparameters 
// number of folds for cross-validation
val numFolds = 10
// number of maximum iteration
val MaxIter: Seq[Int] = Seq(1000)
// regression parameter
val RegParam: Seq[Double] = Seq(0.001)
// the value of tolerance
val Tol: Seq[Double] = Seq(1e-6)
// finally, elastic network parameter
val ElasticNetParam: Seq[Double] = Seq(1)

// Create an LinerRegression estimator
val model = new LinearRegression()
    .setFeaturesCol("features")
    .setLabelCol("label")

// Building the Pipeline for transformations and predictor
println("Building ML pipeline")
val pipeline = new Pipeline().setStages(Preprocessing.stringIndexerStages :+ Preprocessing.assembler :+ model)

// ***********************************************************
println("Preparing K-fold Cross Validation and Grid Search: Model tuning")
// ***********************************************************

val paramGrid = new ParamGridBuilder()
  .addGrid(model.maxIter, MaxIter)
  .addGrid(model.regParam, RegParam)
  .addGrid(model.tol, Tol)
  .addGrid(model.elasticNetParam, ElasticNetParam)
  .build()

val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new RegressionEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(numFolds)

// ************************************************************
println("Training model with Linear Regression algorithm")
// ************************************************************
val cvModel = cv.fit(Preprocessing.trainingData)

// Save the workflow
cvModel.write.overwrite().save("model/Log_LR_model")

// **********************************************************************
println("Evaluating model on train and validation set and calculating RMSE")
// **********************************************************************

val trainPredictionsAndLabels = cvModel.transform(Preprocessing.trainingData).select("label", "prediction")
  .map { case Row(label: Double, prediction: Double) => (label, prediction) }.rdd

val validPredictionsAndLabels = cvModel.transform(Preprocessing.validationData).select("label", "prediction")
  .map { case Row(label: Double, prediction: Double) => (label, prediction) }.rdd  

val trainRegressionMetrics = new RegressionMetrics(trainPredictionsAndLabels)
val validRegressionMetrics = new RegressionMetrics(validPredictionsAndLabels)
val bestModel = cvModel.bestModel.asInstanceOf[PipelineModel]

val results = "\n=====================================================================\n" +
  s"Param trainSample: ${Preprocessing.initialSample}\n" +
  s"Param testSample: ${Preprocessing.initialSample}\n" +
  s"TrainingData count: ${Preprocessing.trainingData.count}\n" +
  s"ValidationData count: ${Preprocessing.validationData.count}\n" +
  s"TestData count: ${Preprocessing.testData.count}\n" +
  "=====================================================================\n" +
  s"Param maxIter = ${MaxIter.mkString(",")}\n" +
  s"Param numFolds = ${numFolds}\n" +
  "=====================================================================\n" +
  s"Training data MSE = ${trainRegressionMetrics.meanSquaredError}\n" +
  s"Training data RMSE = ${trainRegressionMetrics.rootMeanSquaredError}\n" +
  s"Training data R-squared = ${trainRegressionMetrics.r2}\n" +
  s"Training data MAE = ${trainRegressionMetrics.meanAbsoluteError}\n" +
  s"Training data Explained variance = ${trainRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n" +
  s"Validation data MSE = ${validRegressionMetrics.meanSquaredError}\n" +
  s"Validation data RMSE = ${validRegressionMetrics.rootMeanSquaredError}\n" +
  s"Validation data R-squared = ${validRegressionMetrics.r2}\n" +
  s"Validation data MAE = ${validRegressionMetrics.meanAbsoluteError}\n" +
  s"Validation data Explained variance = ${validRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n\n" +
  s"CV params explained: ${cvModel.explainParams}\n" +
  s"GBT params explained: ${bestModel.stages.last.asInstanceOf[LinearRegressionModel].explainParams}\n" +
  "\n=====================================================================\n"
println(results)

    val model_columns = Preprocessing.featureCols
    val stages = bestModel.stages
    val coefficients = stages(2).asInstanceOf[LinearRegressionModel].coefficients.toArray
    val intercept = stages(2).asInstanceOf[LinearRegressionModel].intercept
    val coefficient_value = model_columns zip coefficients
    for (i<- 0 to 12){
        println(coefficient_value(i))
    }


// *****************************************
// Now see if we can predict values in our test data
// Generate predictions using our linear regression model for all features in our test data
println("Run prediction on the test set")

val fullPredictions = cvModel.transform(Preprocessing.testData).cache()

// This basically adds a "prediction" column to our testDF dataframe

// Extract the predictions and the "known" correct labels
val testPredictionsAndLabels = fullPredictions.select("prediction", "label").rdd.map(x => (x.getDouble(0), x.getDouble(1)))

// Print out the predicted and actual values for each point
/*
for (prediction <- testPredictionsAndLabels) {
  println(prediction)
}
*/

val testRegressionMetrics = new RegressionMetrics(testPredictionsAndLabels)
val testresults = "\n=====================================================================\n" +
  s"Test data MSE = ${testRegressionMetrics.meanSquaredError}\n" +
  s"Test data RMSE = ${testRegressionMetrics.rootMeanSquaredError}\n" +
  s"Test data R-squared = ${testRegressionMetrics.r2}\n" +
  s"Test data MAE = ${testRegressionMetrics.meanAbsoluteError}\n" +
  s"Test data Explained variance = ${testRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n"
println(testresults)

Building ML pipeline
Preparing data for before the training
Null values exist in the DataFrame
root
 |-- zipcode: string (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- washer: integer (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- review_scores_rating: integer (nullable = true)
 |-- no_bedrooms: integer (nullable = true)
 |-- amenities: integer (nullable = true)
 |-- property_code: integer (nullable = true)
 |-- room_code: integer (nullable = true)
 |-- bed_type_code: integer (nullable = true)
 |-- cancellation_code: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- log_accom: double (nullable = true)
 |-- log_rating: double (nullable = true)

()
+-------+------------------+-------------------+---------+------------------+-----------------+----------------+
|zipcode|             label|          log_accom|amenities|        log_rating|cancellation_code|availability_365|
+-------+

numFolds: Int = 10
MaxIter: Seq[Int] = List(1000)
RegParam: Seq[Double] = List(0.001)
Tol: Seq[Double] = List(1.0E-6)
ElasticNetParam: Seq[Double] = List(1.0)
model: org.apache.spark.ml.regression.LinearRegression = linReg_87a9d554c717
pipeline: org.apache.spark.ml.Pipeline = pipeline_528acd9e76f2
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linReg_87a9d554c717-elasticNetParam: 1.0,
	linReg_87a9d554c717-maxIter: 1000,
	linReg_87a9d554c717-regParam: 0.001,
	linReg_87a9d554c717-tol: 1.0E-6
})
cv: org.apache.spark.ml.tuning.CrossValidator = cv_0d3a2db0721c
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_0d3a2db0721c
trainPredictionsAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1468] at rdd at <console>:136
validPredictionsAndLabels...

## Log Linear Regression with Interactions

### Preprocessing

In [14]:
object Preprocessing {
    // Shuffling and preparing the data
    var initialSample = 1.0
    val wrangledData = "Assessment_2_Wrangled.csv"
    val spark = SparkSessionCreate.createSession()
    
    import spark.implicits._

    val dataInput = spark.read
        .option("header", "true")
        .schema(customSchema)
        .format("com.databricks.spark.csv")
        .load(wrangledData)
    
    // DATA ARRANGEMENT BEFORE THE SPLIT
    // ============================================
    
    println("Preparing data for before the training")
    val dataArrange1 = dataInput.withColumn("property_code", when(col("property_type") === "house", lit("3").cast(IntegerType))
          .when(col("property_type") === "hotel", lit("2").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange2 = dataArrange1.withColumn("room_code", when(col("room_type") === "Entire home/apt", lit("3").cast(IntegerType))
          .when(col("room_type") === "Private room", lit("2").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange3 = dataArrange2.withColumn("bed_type_code", when(col("bed_type") === "real bed", lit("3").cast(IntegerType))
          .otherwise(lit("1").cast(IntegerType)))

    val dataArrange4 = dataArrange3.withColumn("cancellation_code", when(col("cancellation_policy") === "strict", lit("1").cast(IntegerType))
          .when(col("cancellation_policy") === "moderate", lit("2").cast(IntegerType))
          .otherwise(lit("3").cast(IntegerType)))

    
    // Taking logarithm of selected fields and their INTERACTIONS which are considered in our proposal and EDA work
    val dataLog1 = dataArrange4.withColumn("log_estimated_annual_revenue", log10("estimated_annual_revenue"))
    
    // INTERACTIONS
    // logarithm of interactions
    val dataLog2 = dataLog1.withColumn("log_extra_amenities",  log(($"kitchen"+$"washer")*($"wifi"+$"cable_tv")))    
    val dataLog3 = dataLog2.withColumn("log_review_rating", log10($"review_scores_rating"*$"number_of_reviews")) 
    
    // interactions
    val dataInt1 = dataLog3.withColumn("accom_vs_amenities", $"accommodates"/(($"beds"/$"bedrooms")+$"bathrooms"))
    val dataInt2 = dataInt1.withColumn("property_index", $"property_code"*$"room_code")

    // Filtering the data and remove rows we have defined in our EDA work
    // Step 01:
    val dataFilt = dataInt2.filter(!(dataInt2("no_bedrooms") === 1 && dataInt2("amenities") === 111 && dataInt2("price") > 7500))

    // Step 02:
    var amenitiesList = List(211, 311)
    val dataFiltered = dataFilt.filter(!(dataFilt("no_bedrooms") === 3 && dataFilt("amenities").isin(amenitiesList:_*) && dataFilt("price") > 5000))


    val dataDrop = dataFiltered.drop("property_type", "room_type", "cable_tv","listing_id",
                                     "bed_type", "cancellation_policy", "kitchen", "wifi", "cable_tv", "washer",
                                     "host_response_rate", "city", "estimated_annual_revenue",
                                     "accommodates", "beds", "price", "property_code","room_code",
                                     "bedrooms", "review_scores_rating", "number_of_reviews", "price")    

    var data = dataDrop.withColumnRenamed("log_estimated_annual_revenue", "label").sample(false, initialSample)
    var DF = data.na.drop()

    // Null check
    if (data == DF) println("No null values in the DataFrame")
    else {
        println("Null values exist in the DataFrame")
        data = DF
    }

    println(data.printSchema())
    data.select("zipcode", "label", "property_index", "accom_vs_amenities", "log_extra_amenities",
                "log_review_rating", "availability_365").show()

    // ============================================
    // TWO-STAGE SPLITTING 
    // ============================================
    
    // 1) Split the training data as train (training + validation) and test data
    val seed = 12345L
    val splits01 = data.randomSplit(Array(0.75, 0.25), seed)
    val (trainData, testData) = (splits01(0), splits01(1))               
    
    // 2) Split the train data as training and validation data
    val splits02 = trainData.randomSplit(Array(0.8, 0.2), seed)
    val (trainingData, validationData) = (splits02(0), splits02(1))


    // Caching all data for faster in-memory access
    trainingData.cache
    validationData.cache
    testData.cache
    
    // ============================================
    // DATA PIPELINE ARRANGEMENT
    // ============================================

    // Identify the categorical columns
    def isCateg(c: String): Boolean = c matches "zipcode"

    def categNewCol(c: String): String = if (isCateg(c)) s"idx_${c}" else c

    // Function to select only feature columns (omit id and label)
    def onlyFeatureCols(c: String): Boolean = !(c matches "id|label|zipcode")

    // Definitive set of feature columns
    val featureCols = trainingData.columns
        .filter(onlyFeatureCols)
        .map(categNewCol)

    // StringIndexer for categorical columns (OneHotEncoder should be evaluated as well)
    val stringIndexerStages = trainingData.columns.filter(isCateg)
      .map(c => new StringIndexer()
      .setInputCol(c)
      .setOutputCol(categNewCol(c))
      .fit(dataInput.select(c)))    
    
      // VectorAssembler for training features
      val assembler = new VectorAssembler()
        .setInputCols(featureCols)
        .setOutputCol("features")
    // ============================================    
}

defined object Preprocessing


### Run Pipeline

In [15]:
Logger.getLogger("org").setLevel(Level.OFF)
Logger.getLogger("akka").setLevel(Level.OFF)

// Defining hyperparameters 
// number of folds for cross-validation
val numFolds = 10
// number of maximum iteration
val MaxIter: Seq[Int] = Seq(1000)
// regression parameter
val RegParam: Seq[Double] = Seq(0.001)
// the value of tolerance
val Tol: Seq[Double] = Seq(1e-6)
// finally, elastic network parameter
val ElasticNetParam: Seq[Double] = Seq(1)

// Create an LinerRegression estimator
val model = new LinearRegression()
    .setFeaturesCol("features")
    .setLabelCol("label")

// Building the Pipeline for transformations and predictor
println("Building ML pipeline")
val pipeline = new Pipeline().setStages(Preprocessing.stringIndexerStages :+ Preprocessing.assembler :+ model)

// ***********************************************************
println("Preparing K-fold Cross Validation and Grid Search: Model tuning")
// ***********************************************************

val paramGrid = new ParamGridBuilder()
  .addGrid(model.maxIter, MaxIter)
  .addGrid(model.regParam, RegParam)
  .addGrid(model.tol, Tol)
  .addGrid(model.elasticNetParam, ElasticNetParam)
  .build()

val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new RegressionEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(numFolds)

// ************************************************************
println("Training model with Linear Regression algorithm")
// ************************************************************
val cvModel = cv.fit(Preprocessing.trainingData)

// Save the workflow
cvModel.write.overwrite().save("model/Log_LR_with_interactions_model")

// **********************************************************************
println("Evaluating model on train and validation set and calculating RMSE")
// **********************************************************************

val trainPredictionsAndLabels = cvModel.transform(Preprocessing.trainingData).select("label", "prediction")
  .map { case Row(label: Double, prediction: Double) => (label, prediction) }.rdd

val validPredictionsAndLabels = cvModel.transform(Preprocessing.validationData).select("label", "prediction")
  .map { case Row(label: Double, prediction: Double) => (label, prediction) }.rdd  

val trainRegressionMetrics = new RegressionMetrics(trainPredictionsAndLabels)
val validRegressionMetrics = new RegressionMetrics(validPredictionsAndLabels)
val bestModel = cvModel.bestModel.asInstanceOf[PipelineModel]

val results = "\n=====================================================================\n" +
  s"Param trainSample: ${Preprocessing.initialSample}\n" +
  s"Param testSample: ${Preprocessing.initialSample}\n" +
  s"TrainingData count: ${Preprocessing.trainingData.count}\n" +
  s"ValidationData count: ${Preprocessing.validationData.count}\n" +
  s"TestData count: ${Preprocessing.testData.count}\n" +
  "=====================================================================\n" +
  s"Param maxIter = ${MaxIter.mkString(",")}\n" +
  s"Param numFolds = ${numFolds}\n" +
  "=====================================================================\n" +
  s"Training data MSE = ${trainRegressionMetrics.meanSquaredError}\n" +
  s"Training data RMSE = ${trainRegressionMetrics.rootMeanSquaredError}\n" +
  s"Training data R-squared = ${trainRegressionMetrics.r2}\n" +
  s"Training data MAE = ${trainRegressionMetrics.meanAbsoluteError}\n" +
  s"Training data Explained variance = ${trainRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n" +
  s"Validation data MSE = ${validRegressionMetrics.meanSquaredError}\n" +
  s"Validation data RMSE = ${validRegressionMetrics.rootMeanSquaredError}\n" +
  s"Validation data R-squared = ${validRegressionMetrics.r2}\n" +
  s"Validation data MAE = ${validRegressionMetrics.meanAbsoluteError}\n" +
  s"Validation data Explained variance = ${validRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n\n" +
  s"CV params explained: ${cvModel.explainParams}\n" +
  s"GBT params explained: ${bestModel.stages.last.asInstanceOf[LinearRegressionModel].explainParams}\n" +
  "\n=====================================================================\n"
println(results)

    val model_columns = Preprocessing.featureCols
    val stages = bestModel.stages
    val coefficients = stages(2).asInstanceOf[LinearRegressionModel].coefficients.toArray
    val intercept = stages(2).asInstanceOf[LinearRegressionModel].intercept
    val coefficient_value = model_columns zip coefficients
    for (i<- 0 to 9){
        println(coefficient_value(i))
    }

// *****************************************
// Now see if we can predict values in our test data
// Generate predictions using our linear regression model for all features in our test data
println("Run prediction on the test set")

val fullPredictions = cvModel.transform(Preprocessing.testData).cache()

// This basically adds a "prediction" column to our testDF dataframe

// Extract the predictions and the "known" correct labels
val testPredictionsAndLabels = fullPredictions.select("prediction", "label").rdd.map(x => (x.getDouble(0), x.getDouble(1)))

// Print out the predicted and actual values for each point
/*
for (prediction <- testPredictionsAndLabels) {
  println(prediction)
}
*/

val testRegressionMetrics = new RegressionMetrics(testPredictionsAndLabels)
val testresults = "\n=====================================================================\n" +
  s"Test data MSE = ${testRegressionMetrics.meanSquaredError}\n" +
  s"Test data RMSE = ${testRegressionMetrics.rootMeanSquaredError}\n" +
  s"Test data R-squared = ${testRegressionMetrics.r2}\n" +
  s"Test data MAE = ${testRegressionMetrics.meanAbsoluteError}\n" +
  s"Test data Explained variance = ${testRegressionMetrics.explainedVariance}\n" +
  "=====================================================================\n"
println(testresults)

Building ML pipeline
Preparing data for before the training
Null values exist in the DataFrame
root
 |-- zipcode: string (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- no_bedrooms: integer (nullable = true)
 |-- amenities: integer (nullable = true)
 |-- bed_type_code: integer (nullable = true)
 |-- cancellation_code: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- log_extra_amenities: double (nullable = true)
 |-- log_review_rating: double (nullable = true)
 |-- accom_vs_amenities: double (nullable = true)
 |-- property_index: integer (nullable = true)

()
+-------+------------------+--------------+------------------+-------------------+------------------+----------------+
|zipcode|             label|property_index|accom_vs_amenities|log_extra_amenities| log_review_rating|availability_365|
+-------+------------------+--------------+------------------+-------------------+------------------+------------

numFolds: Int = 10
MaxIter: Seq[Int] = List(1000)
RegParam: Seq[Double] = List(0.001)
Tol: Seq[Double] = List(1.0E-6)
ElasticNetParam: Seq[Double] = List(1.0)
model: org.apache.spark.ml.regression.LinearRegression = linReg_d9175e658861
pipeline: org.apache.spark.ml.Pipeline = pipeline_c6c50ae4444f
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linReg_d9175e658861-elasticNetParam: 1.0,
	linReg_d9175e658861-maxIter: 1000,
	linReg_d9175e658861-regParam: 0.001,
	linReg_d9175e658861-tol: 1.0E-6
})
cv: org.apache.spark.ml.tuning.CrossValidator = cv_da02916ea3dc
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_da02916ea3dc
trainPredictionsAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[2232] at rdd at <console>:139
validPredictionsAndLabels...

In [16]:
// Stop the session
spark.stop()